In [ ]:
"Building a Structured E-Commerce Assistant Using Groq LLM, LangChain, and Pydantic"

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [3]:
from pydantic import BaseModel, Field
from typing import Union
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

import os



# Define structured output
class ProductInfo(BaseModel):
    product_name: str = Field(..., description="Name of the e-commerce product")
    product_details: str = Field(..., description="Short details about the product")
    price_usd: Union[int, float] = Field(..., description="Tentative price in USD")

# Output parser
parser = PydanticOutputParser(pydantic_object=ProductInfo)

# Prompt template
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an intelligent e-commerce assistant."),
    HumanMessagePromptTemplate.from_template(
        """
Given a user question about a product, extract the following fields:
- product_name
- product_details
- price_usd

Respond in the following JSON format:
{format_instructions}

Question: {user_question}
        """
    )
])

prompt_with_parser = chat_prompt.partial(format_instructions=parser.get_format_instructions())

# Use Groq model
llm = ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0,
    groq_api_key=os.environ["GROQ_API_KEY"]
)

# Create chain
chain = LLMChain(llm=llm, prompt=prompt_with_parser, output_parser=parser)

# Example question
question = "Tell me about the Bose QuietComfort headphones and their price."
result = chain.run(user_question=question)

print(result)


/var/folders/83/srl81_4n5rd_qjvvzp5gjzk00000gp/T/ipykernel_19376/2681281555.py:49: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_with_parser, output_parser=parser)
/var/folders/83/srl81_4n5rd_qjvvzp5gjzk00000gp/T/ipykernel_19376/2681281555.py:53: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run(user_question=question)


product_name='Bose QuietComfort Headphones' product_details='Advanced noise-cancelling headphones with comfortable design and long battery life' price_usd=349.95


In [4]:
result = chain.run(user_question="Tell me about Sony WH‑1000XM5 headphones and their price.")
print(result)


product_name='Sony WH-1000XM5 Headphones' product_details='Industry-leading noise cancellation and exceptional sound quality' price_usd=350.0
